In [1]:
import os
os.makedirs("mcpserver",exist_ok=True)

In [8]:
%%writefile mcpserver/mcpserver1.py

from mcp.server.fastmcp import FastMCP
import requests, json
import wikipedia

mcp = FastMCP("TredenceMCP")

@mcp.tool()
async def get_current_weather(city:str)->dict:
    """ this funciton can be used to get current weather information"""
    api_key="6a8b0ac166a37e2b7a38e64416b3c3fe"
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = json.loads(response.content.decode())
    output = {"city":city,"weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'], "unit":"kelvin"
              }
    return output

@mcp.tool()
async def get_wikipedia_summary(query:str)->str:
    response = wikipedia.summary(query)
    return response


if __name__=="__main__":
    mcp.run(transport='streamable-http')


Overwriting mcpserver/mcpserver1.py


In [ ]:
# run mCP server: python mcpserver/mcpserver1.py

## Implement an Agent which connects to the MCP server and fetches the tools

In [9]:
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

In [10]:
client = MultiServerMCPClient({"TredenceMCP":{"url":"http://127.0.0.1:8000/mcp",
                                              "transport":"streamable_http"}})


tools = await client.get_tools()
tools

[StructuredTool(name='get_current_weather', description=' this funciton can be used to get current weather information', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_current_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7452b10176a0>),
 StructuredTool(name='get_wikipedia_summary', args_schema={'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'get_wikipedia_summaryArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x74529ff3ac00>)]

In [11]:
agent = create_react_agent("azure_openai:gpt-4.1-mini",tools)
await agent.ainvoke({"messages":[{"role":'user',"content":"what is the weather in Delhi?"}]})

{'messages': [HumanMessage(content='what is the weather in Delhi?', additional_kwargs={}, response_metadata={}, id='a965841c-686e-444a-a667-7f56077c9833'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jGx007GrI8INRWfkwvwpECmK', 'function': {'arguments': '{"city":"Delhi"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 73, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_178c8d546f', 'id': 'chatcmpl-BeH7kp7VCeDrQvF0kXLFOn5TVVh1H', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': Fals

In [12]:
await agent.ainvoke({"messages":[{"role":'user',"content":"Tell me more about city Jaiselmer?"}]})

{'messages': [HumanMessage(content='Tell me more about city Jaiselmer?', additional_kwargs={}, response_metadata={}, id='a967a7ac-026b-4021-ae06-512eab9c404f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Z7TYbT19VySz0F0kCYWvfzb', 'function': {'arguments': '{"query":"Jaisalmer"}', 'name': 'get_wikipedia_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 76, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_178c8d546f', 'id': 'chatcmpl-BeH8KVa84038T7PrcTnWapQi2Zc2W', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'det